# Создание таблиц с необходимыми данными

## Импорт библиотеки pandas

In [229]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Загрузка датасета и установка года данных для обработки

Компиляция данных так устроена, что необходимо указывать год сбора данных. После этого будут получены таблицы за конкретный год. Если сгенерировать таблицы за все 3 года и запустить программу еще раз, то создадутся таблицы со всеми данными вместе.
Так было сделано, чтобы проводить сбор и просмотр данных раздельно за каждый год так, как позволял функционал jupyter notebook.

In [230]:
year = 2017
df = pd.read_stata(f'InitialData/data{year}.dta', convert_categoricals=False).fillna(-1)
initial_count = df.count()['caseno']

## Импорт собственных часто используемых функций и создание групп вопросов

In [231]:
from utils import reg, rename, calc_groups, lgbt, women, family, parents, children, marriage
family_group, marriage_group, women_group, children_group, parents_group, lgbt_group, gender, age = calc_groups(year)

### Выбор данных 2017 года из датасета для проверки репрезентативности выборки

In [232]:
if year == 2017:
    init_data_of_age_and_sex = df[(df['age'] > 0) & (df[gender[0]] > 0)]

## Оставляем только строки с валидными, интересующими нас значениями

In [233]:
if year == 2017:
    df = df[(df['v32'] >= 0)
            & (df['v2'] >= 0)
            & (df['v225'] >= 0)
            & (df['v71'] >= 0)
            & (df['v155'] >= 0)
            & (df['v72'] >= 0)
            & (df['age_r3'] > 0)
            & (df['v74'] >= 0)
            & (df['v75'] >= 0)
            & (df['v83'] >= 0)
            & (df['v69'] >= 0)
            & (df['v154'] >= 0)
            & (df['v84'] >= 0)
            & (df['v82'] >= 0)
            & (df['v65'] >= 0)
            & (df['v153'] >= 0)]
if year == 2008:
    df = df[(df['v2'] >= 0)
            & (df['v302'] >= 0)
            & (df['v251'] >= 0)
            & (df['v136'] >= 0)
            & (df['v150'] >= 0)
            & (df['v242'] >= 0)
            & (df['v159'] >= 0)
            & (df['v160'] >= 0)
            & (df['v161'] >= 0)
            & (df['v145'] >= 0)
            & (df['v156'] >= 0)
            & (df['v241'] >= 0)
            & (df['v167'] >= 0)
            & (df['v240'] >= 0)
            & (df['v154'] >= 0)]
if year == 1999:
    df = df[(df['v2'] >= 0)
            & (df['v242'] >= 0)
            & (df['v150'] >= 0)
            & (df['v133'] >= 0)
            & (df['v99'] >= 0)
            & (df['v156'] >= 0)
            & (df['v154'] >= 0)
            & (df['v233'] >= 0)
            & (df['v144'] >= 0)
            & (df['v162'] >= 0)
            & (df['v232'] >= 0)
            & (df['v291'] >= 0)]

# Quantity of filtrated values
filtrated_count = initial_count - df.count()['caseno']

## Оставляем только интересующие нас колонки

In [234]:
df = df[
    ['country'] + gender + family_group + marriage_group + women_group + children_group + parents_group + lgbt_group + age]

## Создание индексов

In [235]:
df['region'] = df.country.apply(reg)
df['name'] = df['country'].apply(rename)

In [236]:
df['family'] = df[family_group].apply(family, axis=1, args=[year])
df['marriage'] = df[marriage_group].apply(marriage, axis=1, args=[year])
df['women'] = df[women_group].apply(women, axis=1, args=[year])
df['children'] = df[children_group].apply(children, axis=1, args=[year])
df['parents'] = df[parents_group].apply(parents, axis=1, args=[year])
df['lgbt'] = df[lgbt_group].apply(lgbt, axis=1, args=[year])

In [237]:
df['nsc'] = (df.marriage + df.women + df.children + df.parents + df.lgbt) / 5

## Создание структурированных таблиц с посчитанными данными по годам

In [238]:
from scipy.stats import variation

# Calculation of each stat in dataset depending on parameters
def calc_stat(data, categ, statistic, idx):
    cat = df.country if categ is by_country else df.region
    match idx:
        case 'median':
            return df[cat == data][statistic].median()
        case 'kurtosis':
            return df[cat == data][statistic].skew()
        case 'skew':
            return df[cat == data][statistic].kurt()
        case 'variation':
            return variation(df[cat == data][statistic])

In [239]:
# Creation of datasets
basic_fields = ['nsc', 'lgbt', 'family', 'marriage', 'women', 'children', 'parents']
by_country = df.groupby('country')[
    basic_fields + ['region', 'name']].describe(
    include='all').dropna(axis=1)
by_region = df.groupby('region')[basic_fields].describe(
    include='all').dropna(axis=1)
by_people = df[
    basic_fields + ['region', 'country', 'name'] + gender]

# Creation of specific naming
by_country.columns = by_country.columns.map('_'.join)
by_region.columns = by_region.columns.map('_'.join)

# Specifying year
by_country['Year'] = year
by_region['Year'] = year
by_people['Year'] = year

# Cosmetic changes
by_country.drop(by_country.filter(regex='count').columns, axis=1, inplace=True)
by_country.drop('region_unique', axis=1, inplace=True)
by_country.drop('name_unique', axis=1, inplace=True)
by_country.drop('name_freq', axis=1, inplace=True)
by_country.rename(columns={'region_top': 'region'}, inplace=True)
by_country.rename(columns={'name_top': 'name'}, inplace=True)
by_country.rename(columns={'region_freq': 'num'}, inplace=True)
by_region['num'] = by_region['nsc_count']
by_region.drop(by_region.filter(regex='count').columns, axis=1, inplace=True)
by_people.rename(columns={gender[0]: 'gender'}, inplace=True)

# Calculating statistics
for category in [by_country, by_region]:
    category.reset_index(inplace=True)
    sub_cat = by_country.country if category is by_country else by_region.region
    for stat in ('median', 'kurtosis', 'skew', 'variation'):
        for index in ['nsc', 'lgbt', 'family', 'marriage', 'women', 'children', 'parents']:
            category[f'{index}_{stat}'] = sub_cat.apply(calc_stat, args=[category, stat, index])
    category = category.reindex(sorted(category.columns), axis=1)

# Saving data
by_country.to_csv(f'GeneratedData/ByCountries-{year}.csv', index=False)
by_region.to_csv(f'GeneratedData/ByRegions-{year}.csv', index=True)
by_people.to_csv(f'GeneratedData/ByPeople-{year}.csv', index=True)

C:\Users\Святослав\AppData\Local\Temp\ipykernel_13892\229677063.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  by_people['Year'] = year
C:\Users\Святослав\AppData\Local\Temp\ipykernel_13892\229677063.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  by_people.rename(columns={gender[0]: 'gender'}, inplace=True)


In [240]:
df.to_csv(f'GeneratedData/CompleteDataset-{year}.csv', index=True)

In [241]:
if year == 2017:
    init_data_of_age_and_sex.to_csv(f'GeneratedData/InitAgeSex-{year}.csv', index=True)

## Компиляция данных вне зависимости от года

In [242]:
for name in ('ByCountries', 'ByPeople', 'ByRegions'):
    data_1999 = pd.read_csv(f'GeneratedData/{name}-1999.csv')
    data_2008 = pd.read_csv(f'GeneratedData/{name}-2008.csv')
    data_2017 = pd.read_csv(f'GeneratedData/{name}-2017.csv')
    data_final = pd.concat([data_1999, data_2008, data_2017]).reset_index(drop=True)
    if name in ('ByPeople', 'ByRegions'):
        data_final.drop(data_final.columns[0], axis=1, inplace=True)
    data_final.to_csv(f'GeneratedData/{name}-all.csv', index=True)